# CPSC 325 Data Science Lab League of Legends 
### By Ben Lombardi 

## Introduction and Project Proposal 

At the start of the semester I was very ambitious in what I wanted to get down. Having just taken CPSC 323 I knew wanted to do something with machine learning. Given my passion for video games I had initially resolved to do a machine learning with League of Legends data. My initial proposal for the project was to create a dashboard that displays summary statistics and visualizations at an account level for recent matches. In addition to this dashboard, I had also proposed to develop 3 machine learning models. 
* Predict the winner of the game based on Team Composition with ideally a component that you could use it live as you were loading into a game. 
* Use machine learning to recommend what items to buy for a given champion given your lane opponent. 
* Create a model that predicts whether a champion will get nerfed or buffed next patch given information on their overall performance on the last patch.

I had also proposed to deploy this app on Google App Engine with Flask, BigQuery, the Riot API, and Google Cloud Storage.

When proposing this project, I had thought that creating a model that predicts the winner based on Team Composition would be very easy and would take no time at all letting me move on to working on the other two models. However, this quickly turned out to not be the case. 

## Data gathering and pipeline
I got all of my data directly from the Riot API. I was able to get a development key from Riot with a rate limit of 20 requests per second, and 100 requests per 2 minutes. I was hoping for a bit higher rate when I initially requested the API key, but it ended up working out fine. I built my training dataset back in March by starting with a "seed" player called doublelift who at the time was a former professional player. I then proceeded to request his most recent 100 matches, appended the responses to a list, and created dictionaries that contained all of the players unique ids as well as the unique match ids. Then I moved on to the next player in the dictionary and requested their most recent 100 matches only added the matches, match ids, and players that were not already in my dataset. I did this until I had roughly 60,000 matches. I then split this data into a 85% training and 15% validation split for training my models. 

Then in early May I repeated this process for 1,500 matches to build my training set. 

This files are soted in data_2.json and data3.json which are not currently in the repo due to their size of several gigabytes. However, the can be made available on request. 

## Model Training
Training my models is where I ran into a lot of trouble. I was initially very set on training my model to only include the Team Composition of each team (i.e. what characters were on each team,categorical data) to try and predict the winner. After initial setbacks, my training process consisted on training 4 models in parallel for each iteration, I trained a kNN Classifier, Decision Tree Classifier, Random Forest Classifier, and different iterations of neural networks. I experimented for a long time with this setup and experimented with different numbers of layers, different numbers of neurons in each layer, different activation functions, and different optimizers and learning rates. I also experimented with autoencoders and convolutional networks. All of these experiments still yielded 50% accuracy across all 4 classifiers, the same as if I had just used a dummy predictor. 

So I decided to add in more data on the characters as inputs. Each character can be described as belonging to one or more classes in the game - "Fighter, Assassin, Mage, Marksmen, Support, and Tank" - and so I decided to map each character's id onto their main class and include that as input as well in an effort to boost performance. However, this had almost zero impact on performance, and only boosted accuracy by a fraction of a percent in most cases. 

Despite my best efforts, as I was not able to get this working, I relaxed my assumption and used other information from my dataset to try and predict the winner. I ended up using the categories "Baron First", "Dragon First", "Rift Herald First". These are all important objectives and can have a very large impact on the game. I did not include the features related to how many of these objectives each team took, each team's total kills, or each team's total experience as this made the prediction task trivially easily and I thought was giving it the model too much information on the match. So I made three models, one for each objective that used said objective and each team's team composition.

Doing this gave me a good amount of success and as a result I made a fourth model that included all of the possible objectives and which team took them first. I ended up deploying a neural network for each model, except for the rift herald model which I used a Random forest classifier. During my training of Neural networks, I found the key factor to be changing the activation function to "sigmoid" and I got significantly better results training using this. 

Lastly, I did spend some time working on the proposed third model. To create this dataset I got the win rates, pick rates, ban rates, and trends for each patch last year off of [Kaggle](https://www.kaggle.com/datasets/vivovinco/league-of-legends-champion-stats) and then I assembled by hand which champs got buffed and nerfed each patch. This ended up being a pretty small dataset of about 5,500 with only 7% of it having a class label of instance. I could not get this model to work no matter what I tried and features that I included. I could not get it to function better than a dummy predictor. I tried regular neural networks, made convolutional networks, and tried windowing my data. To better understand what was going on I tried both linear and logistic regression on this dataset. For Linear Regression I got an adjusted R-squared value of 0 and none of the variables coefficients were statistically significant. With the Logistic Regression, I got an adjusted R-squared value of -0.003 and none of the variables were statistically significant. 

## Model Testing 
Testing my models on my updated Testing set of recent data I got the following accuracies: 
* Team Comp + Baron: 68.9%
* Team Comp + Dragon: 63.9%
* Team Comp + Rift Herald: 57.6%
* Objectives: 81.2% 

Not only are all of these models above 50% accuracy, but they all have higher accuracy than if you predicted the accuracy based on the dominant input (i.e. for the Baron model predicting winner based on who gets baron first). The Baron model had roughly 20% improvement in accuracy over dominant input feature. The Dragon and Rift Herald Model had about 3% improvement in accuracy over dominant input feature. 

It should also be noted these models did worse on the Testing set than on the training set or validation sets. Additionally, the distribution of what objectives had been taken between the Training/Validation set and the Testing set where not the same and generally the Testing set had less objectives taken in it. My main hypothesis of this is the time difference in the data. A good chunk of the data was from recent patches where they have had some rather large balance patches which can substantially change the way the game is played and what players focus on. For future work on this project, think that pulling training data more frequently and keeping a moving window of training data as you remove old data as new data comes in. 

## Deployment 

For the mid-project demo I put a lot work into learning flask and deploying a demo onto Google App Engine. However, I was having a lot of problems getting visualizations onto the flask app on the Google Cloud, so I ended up switching to Streamlit which led to things becoming much smoother. 

## Conclusion

In conclusion, while I did not quite achieve what I initially set out to achieve. I ended pivoting the project and machine learning models into something that I am really quite proud of that stays with the spirit of what I intended the project to be. A description of the repo and scripts can be found in the README. 
 



